In [1]:
#| default_exp nb_state

In [2]:
#| export
from __future__ import annotations

# Notebook state

> Programmatic access to notebook cells and outputs.



# Prologue

In [3]:
#| export
import json
import sys
from datetime import datetime
from inspect import Parameter
from typing import Mapping
from typing import overload
from typing import Protocol
from typing import runtime_checkable
from typing import SupportsIndex

import fastcore.all as FC
from fastcore.foundation import L
from IPython.core.getipython import get_ipython
from IPython.display import display
from IPython.display import HTML
from olio.basic import AD
from olio.basic import bundle_path
from olio.basic import update_


In [4]:
import os
os.environ['DEBUG_BRIDGET'] = 'True'

In [5]:
#| export
import bridget.nb_hooks
from bridget.bridge import BridgePlugin
from bridget.bridge import get_bridge
from bridget.bridge_plugins import NBHooksPlugin
from bridget.bridge_widget import blocks
from bridget.bridge_widget import bundled
from bridget.helpers import bridge_cfg
from bridget.helpers import id_gen
from bridget.helpers import kounter
from bridget.helpers import ms2str
from bridget.nb import NB
from bridget.nb import NBCell


In [6]:
import os
from pathlib import Path
import urllib.parse

from fastcore.test import *
from olio.common import setup_console
from olio.common import shortens


In [7]:
from bridget.bridge_widget import ablocks
from bridget.helpers import DEBUG
from bridget.helpers import in_vscode_notebook


----


In [8]:
#| exporti

BUNDLE_PATH = bundle_path(__name__)
_EMPTY = Parameter.empty
EmptyT = type[_EMPTY]
new_id = id_gen()

In [9]:
console, cprint = setup_console(120)
IDISPLAY = display
print(f"{DEBUG()}")
bridge_cfg

True


{'auto_show': False, 'auto_mount': False, 'auto_id': False, 'bundle_cfg': {'out_dir': [Path('/Users/vic/dev/repo/project/bridget/bridget/js')], 'rewrite_imports': True, 'import_name': 'brdimport'}, 'bootstrap': False, 'current_did': None}

In [10]:
bridge_cfg.auto_show = True
bridge_cfg.auto_id = False

In [11]:
bridge = get_bridge(show_logger=True, wait=5)

moduleName='./bcanvas.js' buffers=[]
moduleName='./fcanvas.js' buffers=[]
moduleName='./bridge.js' buffers=[]
moduleName='./observer.js' buffers=[]
moduleName='./nbstate.js' buffers=[]


----

# NBStateFeedback
> Simple visual feedback of notebook state. We manage the notebook state In the front-end and use this helper to show a little feedback of state changes.

If yuou're reading this in VSCode-ish environment, ensure the extension developed in [nbinspect-vscode](../packages/nbinspect-vscode/README.md) is enabled.  

If you want to know how the notebook's state gets here in real-time, take a look at the [packages](../packages) folder.

In [12]:
#| export

NBSTATE_MIME = 'application/x-notebook-state'

class _NBStateFeedback:
    def __init__(self, **options):
        self.options = AD(feedback=True, hide=False, debug=True, **options)
        self.dh = None; self.shown = False
    def show(self, hide=False, **options):
        if self.dh: self.dh.update(HTML(''))
        self.dh = display({NBSTATE_MIME: update_(self.options, **options, hide=hide)}, raw=True, display_id=True)
        self.shown = True
    def update(self, **options):
        if self.dh: self.dh.update({NBSTATE_MIME: {**update_(self.options, **options), 'id': kounter('nbstate')}}, raw=True)
    def hide(self):
        if self.dh: self.dh.update({NBSTATE_MIME: {**update_(self.options, hide=True), 'id': kounter('nbstate')}}, raw=True)


NBStateFeedback = _NBStateFeedback()

In [13]:
NBStateFeedback.show()

Note that the state data **is** still in the **front-end** (JS-land). This function is just a visual feedback of state changes, but we haven't transferred yet the state to the **kernel** (python-land). We need to have something (the bridge) in the front-end that can transfer the state to the kernel. For that, see next sections.

**VSCode weirdness (again)**: Bridget uses an extension to capture notebook state. In Jupyter environments, the extension is triggered when a notebook is opened, monitors state changes, and sends notifications to the frontend, waiting for a widget to retrieve them. Check the JavaScript console to see the extension's traces.

In VSCode, the extension is also activated when a notebook is opened. However, the front end of the extension—Renderer in VSCode Notebook Extension API parlance—is isolated within a webview, an iframe-like object that the extension can only communicate with via the a very limited messaging API. Renderers are the only means by which extensions can communicate with the JavaScript running in the webview. And renderers can only be triggered by a cell displaying specific MIME types at least once. Therefore, since VSCode only renders its output when a visible cell is executed, the extension's renderer can only obtain the notebook state relayed by the main extension when the cell is run interactively or is already in the notebook and visible. Fortunately, this only needs to happen once; you can delete the cell or its output, and the extension will continue to work. VSCode's foibles.

Keep this in mind when running commands like `Execute Above Cells` or `Execute All Cells` or similar. If the above cell is already displaying its output, that's fine; if not, the extension renderer won't be triggered until it appears, the state won't be passed to the front end, and the rest of the notebook won't work properly.

In [14]:
print(19)

19


After executing above cell, you've noticed a pink flash. We're monitoring state changes in real-time.

Clear the output of the above cell. Open the details and look for cell #19 (if you haven't added any new cells above it). Check that it doesn't have any outputs.

Run the cell again and check to see that it now has outputs.

Create a new cell below this, source or markdown, and write something. Notice that we're updating the state of cell inputs not just outputs.

In [15]:
NBStateFeedback.update(feedback=False)

With feedback disabled, the details disclosure in not shown.

In [16]:
NBStateFeedback.update(feedback=True)

In [17]:
NBStateFeedback.update(debug=False)

`debug` affects javascript dev console logging.

In [18]:
print(29)

29


In [19]:
NBStateFeedback.update(debug=True)

In [20]:
NBStateFeedback.show()

In [21]:
NBStateFeedback.hide()

Though we've hided the feedback output, Bridget is still monitoring and capturing state changes, as you can see in the dev console. In fact, once activated by showing the feedback output, it's not possible to deactivate it.

# NBStateFetcher
> A bridge plugin to retrieve notebook state from the front-end to `bridge.state`.

Simple plugin to grab the notebook state from the front-end and return it to Python where we really want and need it. And where it should have always been, imho. How much unnecessary pain has the MVC pattern done!

In [22]:
#| export

nbstate_js = BUNDLE_PATH / 'js/nbstate.js'

In [23]:
bundled(nbstate_js)();

In [24]:
#| export

class NBStateFetcher(BridgePlugin):
    src = bundled('''
import { initializeNBState } from './nbstate.js';
export default async function initializeFetcher(bridge) {
    const cleanup = initializeNBState(bridge);
    return () => cleanup();
}
''')()

    ctx_name = 'fetcher'

    def __init__(self):
        self.feedback = True; self.debug = True; self._renderer = True
        self._last_update = []
        super().__init__()

    @property
    def opts(self): return {'feedback': self.feedback, 'debug': self.debug}

    def update(self, timeout: float|None=None, **kwargs):
        kwargs['id'] = new_id(self)
        self.send({'ctx': self.ctx_name, 'cmd': 'get_state', **kwargs}, timeout=timeout)

    async def aupdate(self, timeout: float=3, **kwargs):
        kwargs['id'] = new_id(self)
        await self.asend({'ctx': self.ctx_name, 'cmd': 'get_state', **kwargs}, timeout=timeout)

    def on_init(self, *args, info:str, **kwargs):
        super().on_init(*args, info=info, **kwargs)
        if info == 'initialized':
            self.bridge.state = bridget.nb_hooks.get_nb_from_hooks()
            if self._renderer:
                self.log('Requesting initial notebook state...')
                self.update(update='full')
        elif info == 'renderer not found':
            self._renderer = False
            self.log("Renderer not found: NBState is empty. Try calling update(update='full') again...")
    
    def on_state_update(self, *args, state:dict|str, **kwargs):
        d = (json.loads(state) if isinstance(state, str) else state or {})
        self.log(f"State update - type: {d['type']} ts: {ms2str(float(d['timestamp'])/1000)}")
        self._last_update.append(d)
        if d['type'] == 'state':
            self.log(f"---- #cells: {len(d.get('cells', []))}")
            self.bridge.state.setup(**d)
        else:
            if len(self.bridge.state.cells) == 0: return
            try:
                self.bridge.state.apply_diffsMessage(d)
                diffs:list[dict] = d.get('changes', [])
                # self.log(f"#diffs: {len(diffs)}")
                for i, d in enumerate(diffs): self.log(
                    f"---- {i} - cells: {[c['idx'] for c in d['cells']]} "
                    f"added: {[c['idx'] for c in d.get('added', [])]} "
                    f"removed: {d.get('removed', [])}")
            except Exception as e:
                self.log(f"Error applying diffs: {e}")

In [25]:
bridge.logger.show(clear=True)

In [26]:
bridge.add_plugins(fetcher := NBStateFetcher(), wait=3)

test_is('fetcher' in bridge.plugins, True)
test_eq(len(bridge.state.cells), 0)

WARNING: the extension debounces state notifications to avoid excesive throughput. When running `Execute Above Cells` or `Execute All Cells` or similar commands, the extension will group state notifications and the cell below will probably fail. If that's the case, simply run the cell again.

In [27]:
# cell 38
test_eq(len(bridge.state.cells), 78)
uri = urllib.parse.urlparse(bridge.state.nbData['notebookUri'])
if in_vscode_notebook(globals()): test_eq(uri.path, __vsc_ipynb_file__)  # type: ignore
bridge.state[38]

{'idx': 38,
 'cell_type': 'code',
 'source': "# cell 38\ntest_eq(len(bridge.state.cells), 78)\nuri = urllib.parse.urlparse(bridge.state.nbData['notebookUri'])\nif in_vscode_notebook(globals()): test_eq(uri.path, __vsc_ipynb_file__)  # type: ignore\nbridge.state[38]",
 'id': 'X53sZmlsZQ==',
 'metadata': {'brd': {'id': 'd3c0c32b-2b59-4636-8545-711899a84b39'}},
 'outputs': [],
 'execution_count': None}

Note after running previous cell, the state has not yet been updated. Check that it has no outputs or the outputs are old. We'll see how to access the state of the current cell below.

In [28]:
# cell 40
bridge.state.cells[38]

{'idx': 38,
 'cell_type': 'code',
 'source': "# cell 38\ntest_eq(len(bridge.state.cells), 78)\nuri = urllib.parse.urlparse(bridge.state.nbData['notebookUri'])\nif in_vscode_notebook(globals()): test_eq(uri.path, __vsc_ipynb_file__)  # type: ignore\nbridge.state[38]",
 'id': 'X53sZmlsZQ==',
 'metadata': {'brd': {'id': 'd3c0c32b-2b59-4636-8545-711899a84b39'}},
 'outputs': [{'output_type': 'display_data',
   'data': {'text/html': '<style>details ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } details .string { color: #24837b; } details .string::before { content: "\'"; } details .string::after { content: "\'"; } details .number { color: #ad8301; } details .true { color: blue; } details .false { color: red; } details .null { color: gray; } span.n { color: darkgrey; } </style>\n<details open><summary>NBCell@38\n</summary>  <ul>\n    <li>\n<span><span class="n">idx</span>: <span class="v number">38</span></span>    </li>\n    <li>\n<span><span class="n">cell_type</span>: <span class="v string">code</span></span>    </li>\n    <li>\n<span><span class="n">source</span>: <span class="v string"># cell 38\ntest_eq(len(bridge.state.cells), 78)\nuri = urllib.parse.urlparse(bridge.state.nbData[&#x27;notebookUri&#x27;])\nif in_vscode_notebook(globals…</span></span>    </li>\n    <li>\n<span><span class="n">id</span>: <span class="v string">X53sZmlsZQ==</span></span>    </li>\n    <li>\n<span><span class="n">metadata</span>: <span class="v ">{&#x27;brd&#x27;: {&#x27;id&#x27;: &#x27;d3c0c32b-2b59-4636-8545-711899a84b39&#x27;}}</span></span>    </li>\n<details open><summary>outputs\n</summary>      <ul></ul>\n</details>    <li>\n<span><span class="n">execution_count</span>: <span class="v null">None</span></span>    </li>\n  </ul>\n</details>',
    'text/markdown': '> code\n\n```json\n{\n  \'idx\': 38,\n  \'cell_type\': \'code\',\n  \'source\': "# cell 38\\ntest_eq(len(bridge.state.cells), 78)\\nuri = urllib.parse.urlparse(bridge.state.nbData[\'notebookUri\'])\\nif in_vscode_notebook(globals()): test_eq(uri.path, __vsc_ipynb_file__)  # type: ignore\\nbridge.state[38]",\n  \'id\': \'X53sZmlsZQ==\',\n  \'metadata\': {\'brd\': {\'id\': \'d3c0c32b-2b59-4636-8545-711899a84b39\'}},\n  \'outputs\': [],\n  \'execution_count\': None\n}\n```',
    'text/plain': '{\'idx\': 38,\n \'cell_type\': \'code\',\n \'source\': "# cell 38\\ntest_eq(len(bridge.state.cells), 78)\\nuri = urllib.parse.urlparse(bridge.state.nbData[\'notebookUri\'])\\nif in_vscode_notebook(globals()): test_eq(uri.path, __vsc_ipynb_file__)  # type: ignore\\nbridge.state[38]",\n \'id\': \'X53sZmlsZQ==\',\n \'metadata\': {\'brd\': {\'id\': \'d3c0c32b-2b59-4636-8545-711899a84b39\'}},\n \'outputs\': [],\n \'execution_count\': None}'},
   'metadata': {'transient': {}, 'bridge': {'captured': True}}}],
 'execution_count': 27}

In [29]:
bridge.logger.show()

In [30]:
print('test5')

test5


In [31]:
display(HTML('<div>test6</div>'))

# NBState
> Simple wrapper around `NB` + `NBStateFetcher` & `NBHooks` bridge plugins

In [32]:
#| export

@runtime_checkable
class StateProvider(Protocol):
    @property
    def state(self) -> NB:...


class NBState(FC.GetAttr):
    _default = 'state'
    def __init__(self, source: StateProvider|Mapping|None=None, *bridge_args, plugins=None, **bridge_kw):
        self._state = None
        if source is None:
            if not NBStateFeedback.shown: NBStateFeedback.show(hide=True)
            plugins = [NBHooksPlugin(), NBStateFetcher()]
            if 'wait' not in bridge_kw: bridge_kw['wait'] = 3
            source = get_bridge(*bridge_args, plugins=plugins, **bridge_kw)
        elif isinstance(source, Mapping): self._state = NB(**source)
        self.source = source

    @overload
    def __getitem__(self, key: SupportsIndex|str, /) -> NBCell: ...
    @overload
    def __getitem__(self, key: slice, /) -> L: ...
    def __getitem__(self, key) -> NBCell|L: 
        if isinstance(key, str):
            cc = self.cells.filter(lambda c: c.metadata['cell_id'] == key)
            return cc[0] if len(cc) else L()  # type: ignore
        return self.cells[key]  # type: ignore

    @property
    def state(self) -> NB:
        return self.source.state if self._state is None else self._state  # type: ignore

    def this(self: NBState, idx:int|None=None) -> NBCell:...

In [33]:
state = json.loads(Path('../packages/nbinspect-vscode/test/outputs.json').read_text('utf-8'))

nb = NBState(state)

In [34]:
display(nb.mds)
nb.codes

(#2) [0,23]

(#40) [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20...]

In [35]:
nb = NBState(show_logger=True, wait=5)
test_eq(nb.source, get_bridge())

In [36]:
if len(nb.cells): display(nb[50])

{'idx': 50,
 'cell_type': 'code',
 'source': "list(shortens(nb[40:50], 'r', 80))",
 'id': 'Y101sZmlsZQ==',
 'metadata': {'brd': {'id': '9462c3d4-0a55-4829-b867-1a4e2650d232'}},
 'outputs': [],
 'execution_count': None}

In [37]:
list(shortens(nb[40:50], 'r', 80))

["{'idx': 40, 'cell_type': 'code', 'source': '# cell 40\\nbridge.state.cells[38]', …",
 "{'idx': 41, 'cell_type': 'code', 'source': 'bridge.logger.show()', 'id': 'X56sZm…",
 '{\'idx\': 42, \'cell_type\': \'code\', \'source\': "print(\'test5\')", \'id\': \'X60sZmlsZQ==…',
 '{\'idx\': 43, \'cell_type\': \'code\', \'source\': "display(HTML(\'<div>test6</div>\'))", …',
 "{'idx': 44, 'cell_type': 'markdown', 'source': '# NBState\\n> Simple wrapper arou…",
 '{\'idx\': 45, \'cell_type\': \'code\', \'source\': "#| export\\n\\n@runtime_checkable\\ncla…',
 '{\'idx\': 46, \'cell_type\': \'code\', \'source\': "state = json.loads(Path(\'../packages…',
 "{'idx': 47, 'cell_type': 'code', 'source': 'display(nb.mds)\\nnb.codes', 'id': 'X…",
 "{'idx': 48, 'cell_type': 'code', 'source': 'nb = NBState(show_logger=True, wait=…",
 "{'idx': 49, 'cell_type': 'code', 'source': 'if len(nb.cells): display(nb[50])', …"]

In [38]:
list(shortens(nb.find('class').attrgot('source'), 'r', 80))

["#| export\n\nNBSTATE_MIME = 'application/x-notebook-state'\n\nclass _NBStateFeedback…",
 "#| export\n\nclass NBStateFetcher(BridgePlugin):\n    src = bundled('''\nimport { in…",
 '#| export\n\n@runtime_checkable\nclass StateProvider(Protocol):\n    @property\n    d…',
 "list(shortens(nb.find('class').attrgot('source'), 'r', 80))"]

In [39]:
# find_me

In [40]:
#| hide
list(shortens(nb.find('# find_me'), 'r', 80))

["{'idx': 52, 'cell_type': 'code', 'source': '# find_me', 'id': 'Y103sZmlsZQ==', '…",
 '{\'idx\': 53, \'cell_type\': \'code\', \'source\': "#| hide\\nlist(shortens(nb.find(\'# fi…',
 '{\'idx\': 54, \'cell_type\': \'code\', \'source\': "#| hide\\nnb.find(\'# find_me\').filter…']

In [41]:
#| hide
nb.find('# find_me').filter(lambda x: 'hide' not in x.directives_).attrgot('source')

(#1) ['# find_me']

In [42]:
# this cell2
# nb.update()
nb.find('# this cell2').attrgot('source')

(#1) ["# this cell2\n# nb.update()\nnb.find('# this cell2').attrgot('source')"]

# Cell id

For this to work, bridge must be loaded with the `NBHooksPlugin` (NBState takes care of all that) .

In [43]:
# really hate stupid wiggly reds
__cellinfo__:AD

In [44]:
print(f"{__cellinfo__.cell_id!r}\n{__cellinfo__.source!r}")
test_eq(__cellinfo__.source[:29], 'print(f"{__cellinfo__.cell_id')

'Y112sZmlsZQ=='
'print(f"{__cellinfo__.cell_id!r}\\n{__cellinfo__.source!r}")\ntest_eq(__cellinfo__.source[:29], \'print(f"{__cellinfo__.cell_id\')'


In [45]:
# cell x
idx = nb.find('# cell x')[0].idx  # type: ignore
test_eq(__cellinfo__.cell_id, nb[idx].id)
nb[idx]

{'idx': 59,
 'cell_type': 'code',
 'source': "# cell x\nidx = nb.find('# cell x')[0].idx  # type: ignore\ntest_eq(__cellinfo__.cell_id, nb[idx].id)\nnb[idx]",
 'id': 'Y113sZmlsZQ==',
 'metadata': {'brd': {'id': 'd77063e2-e8eb-49f8-a062-11f66ba86f9f'}},
 'outputs': [],
 'execution_count': None}

# this

`this` marks the finale of the first part of Bridget.

We now have all the pieces to build the bridge.
- fasthtml & other helper scripts
- loader of arbitrary JS code
- notebook state fetcher
- nbstate and nb hooks

Next steps will be to add tools to edit notebook outputs.

In [46]:
#| export

FIRST = -sys.maxsize
LAST = sys.maxsize

# this: Literal['this'] = 'this'

In [47]:
#| export

def this(idx:int|None=None) -> NBCell:
    "Current cell if `idx` is None, or cell at `idx` from current cell upwards. Raises if not found."
    # if not ((bridge := get_bridge()).plugins.get('fetcher', None)):
    #     bridge.add_plugins(fetcher := NBStateFetcher(), wait=3)
    #     blocks(lambda: fetcher.is_initialized is not None, 3)
    # Ummm, don't really need the `fetcher` for this to work
    if not ((bridge := get_bridge()).plugins.get('nbhooks', None)): bridge.add_plugins(NBHooksPlugin())
    shell = get_ipython()
    if shell is None: raise IndexError('No IPython shell found')
    cellinfo = shell.user_ns.get('__cellinfo__')
    if not cellinfo: raise IndexError('__cellinfo__ is not set')
    this_cell = bridge.state[cellinfo.cell_id]
    this_cell.source = cellinfo.source
    if not idx: return this_cell
    this_idx, cells = this_cell.idx, bridge.state.cells
    return cells[
        max(0, this_idx + idx)] if idx < 0 else cells[min(len(cells)-1, this_idx + idx)]  # type: ignore

In [48]:
bridge.logger.show(clear=True)

In [49]:
c = this(); c

{'idx': 64,
 'cell_type': 'code',
 'source': 'c = this(); c',
 'id': 'Y121sZmlsZQ==',
 'metadata': {'brd': {'id': 'a8d3529d-06b6-4d95-8a62-ea664ec0d52e'}},
 'outputs': [],
 'execution_count': None}

In [50]:
c

{'idx': 64,
 'cell_type': 'code',
 'source': 'c = this(); c',
 'id': 'Y121sZmlsZQ==',
 'metadata': {'brd': {'id': 'a8d3529d-06b6-4d95-8a62-ea664ec0d52e'}},
 'outputs': [{'output_type': 'execute_result',
   'execution_count': 49,
   'data': {'text/plain': "{'idx': 64,\n 'cell_type': 'code',\n 'source': 'c = this(); c',\n 'id': 'Y121sZmlsZQ==',\n 'metadata': {'brd': {'id': 'a8d3529d-06b6-4d95-8a62-ea664ec0d52e'}},\n 'outputs': [],\n 'execution_count': None}",
    'text/html': '<style>details ul { list-style-type:none; list-style-position: outside; padding-inline-start: 22px; margin: 0; } details .string { color: #24837b; } details .string::before { content: "\'"; } details .string::after { content: "\'"; } details .number { color: #ad8301; } details .true { color: blue; } details .false { color: red; } details .null { color: gray; } span.n { color: darkgrey; } </style>\n<details open><summary>NBCell@64\n</summary>  <ul>\n    <li>\n<span><span class="n">idx</span>: <span class="v number">64</span></span>    </li>\n    <li>\n<span><span class="n">cell_type</span>: <span class="v string">code</span></span>    </li>\n    <li>\n<span><span class="n">source</span>: <span class="v string">c = this(); c</span></span>    </li>\n    <li>\n<span><span class="n">id</span>: <span class="v string">Y121sZmlsZQ==</span></span>    </li>\n    <li>\n<span><span class="n">metadata</span>: <span class="v ">{&#x27;brd&#x27;: {&#x27;id&#x27;: &#x27;a8d3529d-06b6-4d95-8a62-ea664ec0d52e&#x27;}}</span></span>    </li>\n<details open><summary>outputs\n</summary>      <ul></ul>\n</details>    <li>\n<span><span class="n">execution_count</span>: <span class="v null">None</span></span>    </li>\n  </ul>\n</details>',
    'text/markdown': "> code\n\n```json\n{\n  'idx': 64,\n  'cell_type': 'code',\n  'source': 'c = this(); c',\n  'id': 'Y121sZmlsZQ==',\n  'metadata': {'brd': {'id': 'a8d3529d-06b6-4d95-8a62-ea664ec0d52e'}},\n  'outputs': [],\n  'execution_count': None\n}\n```"},
   'metadata': {}}],
 'execution_count': 49}

`this` is meant to be used interactively (as most of Bridget is).

The main function of `this` is to get the current cell, or more so, the current cell's index, and get your bearings in the current notebook structure.

Note that due to the client-server nature of the Jupyter notebooks, code running in the kernel can't possibly access the cell currently being executed. The cell state is maintained byt the front-end, not the kernel. The kernel in fact knows nothing of notebooks or their structure. We can aspire at most to get the cell's index and source. The output is not yet determined, though Bridget will capture it and you can access it **after** the cell has run, not during the cell execution. You can find the cell with NBState afterwards.

Also be aware, as always, that batch commands like "Execute Above Cells" and similar are implemented very differently by Jupyter Notebooks vendors. The cells in the run queue may start running in order (or not, I've seen ships on fire off the shoulder of Orion), but not necessarily finish up in that order. Bridget may receive the cells in any order and therefore, the notebook state is not settled until the end. In general, in Jupyter Notebooks, cells that depends on outputs of other cells will surely fail when not running interactively. This is known issue (and a desing flaw imo, that can be resolved simply by the front-end sending the cells in order to the kernel and waiting for each one to fisnish before sending the next one) and one of the reason Marimo and other Jupyter modern alternatives exists.

In [51]:
this(-1)

{'idx': 66,
 'cell_type': 'markdown',
 'source': '`this` is meant to be used interactively (as most of Bridget is).\n\nThe main function of `this` is to get the current cell, or more so, the current cell\'s index, and get your bearings in the current notebook structure.\n\nNote that due to the client-server nature of the Jupyter notebooks, code running in the kernel can\'t possibly access the cell currently being executed. The cell state is maintained byt the front-end, not the kernel. The kernel in fact knows nothing of notebooks or their structure. We can aspire at most to get the cell\'s index and source. The output is not yet determined, though Bridget will capture it and you can access it **after** the cell has run, not during the cell execution. You can find the cell with NBState afterwards.\n\nAlso be aware, as always, that batch commands like "Execute Above Cells" and similar are implemented very differently by Jupyter Notebooks vendors. The cells in the run queue may start running in order (or not, I\'ve seen ships on fire off the shoulder of Orion), but not necessarily finish up in that order. Bridget may receive the cells in any order and therefore, the notebook state is not settled until the end. In general, in Jupyter Notebooks, cells that depends on outputs of other cells will surely fail when not running interactively. This is known issue (and a desing flaw imo, that can be resolved simply by the front-end sending the cells in order to the kernel and waiting for each one to fisnish before sending the next one) and one of the reason Marimo and other Jupyter modern alternatives exists.',
 'id': 'Y123sZmlsZQ==',
 'metadata': {'brd': {'id': 'a2dc2fec-bc27-4b20-a96b-ae35dda0525f'}}}

In [52]:
this(-3).source

'c'

In [53]:
this(FIRST).source

'#| default_exp nb_state'

In [54]:
this(LAST).source

"if FC.IN_NOTEBOOK:\n    BUNDLE_PATH = bundle_path(__name__)\n    for f in ['nbstate']: bundled(BUNDLE_PATH / f'js/{f}.js')()\n    nb_path = '21_nb_state.ipynb'\n    # nbdev_clean(nb_path)\n    nbdev.nbdev_export(nb_path)"

In [55]:
this(1).source

'# get_nb'

# get_nb

In [56]:
#| export

__nbstate__ = None

@FC.delegates(get_bridge)  # type: ignore
def get_nb(*args, show_feedback:bool=False, **kwargs):
    global __nbstate__
    if __nbstate__ is None:
        wait=kwargs.pop('wait', 5)
        __nbstate__ = NBState(*args, wait=wait, **kwargs)
    else: get_bridge(*args, **kwargs)
    if show_feedback: NBStateFeedback.show()
    return __nbstate__

In [57]:
#| export

if bridge_cfg.bootstrap: get_nb(show_logger=True)

# Colophon
----


In [58]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean
from olio.basic import bundle_path
from bridget.bridge_widget import bundled

In [59]:
if FC.IN_NOTEBOOK:
    BUNDLE_PATH = bundle_path(__name__)
    for f in ['nbstate']: bundled(BUNDLE_PATH / f'js/{f}.js')()
    nb_path = '21_nb_state.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)